In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F

## Autograd

In [3]:
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad= True)

In [11]:
print(a.grad)
print(a)


None
tensor([2., 3.], requires_grad=True)


In [4]:
q = 3*a**3 - b**2

In [12]:
q

tensor([-12.,  65.], grad_fn=<SubBackward0>)

In [13]:
external_grad = torch.tensor([1., 1.])
q.backward(gradient=external_grad)

In [18]:
print(a.grad)
print(b.grad)
print(q.retain_grad())

print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor([36., 81.])
tensor([-12.,  -8.])
None
tensor([True, True])
tensor([True, True])


## nn.Module

In [ ]:
class Model(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)

### Exploring inside a model

In [20]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(2, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 2)  # Output layer for binary classification

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [21]:
# Instantiate the model
model = SimpleNN()
print(model)

SimpleNN(
  (fc1): Linear(in_features=2, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
)


In [22]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [23]:
# Create synthetic dataset
X, y = make_moons(n_samples=1000, noise=0.1, random_state=42)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [24]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [25]:
num_epochs = 100
batch_size = 32

for epoch in range(num_epochs):
    model.train()
    permutation = torch.randperm(X_train.size()[0])
    
    for i in range(0, X_train.size()[0], batch_size):
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = X_train[indices], y_train[indices]

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

    # Print loss every 10 epochs
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 0.1335
Epoch [20/100], Loss: 0.0227
Epoch [30/100], Loss: 0.0095
Epoch [40/100], Loss: 0.0040
Epoch [50/100], Loss: 0.0053
Epoch [60/100], Loss: 0.0061
Epoch [70/100], Loss: 0.0180
Epoch [80/100], Loss: 0.0043
Epoch [90/100], Loss: 0.0027
Epoch [100/100], Loss: 0.0080


In [26]:
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = (predicted == y_test).sum().item() / y_test.size(0)
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 100.00%


In [50]:
print(model, '\n  ------------------ \n')
print(model.fc1, '\n  ------------------ \n')
for param in model.parameters():
    print(param.size())
    print(param.requires_grad)


SimpleNN(
  (fc1): Linear(in_features=2, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
) 
  ------------------ 

Linear(in_features=2, out_features=16, bias=True) 
  ------------------ 

torch.Size([16, 2])
True
torch.Size([16])
True
torch.Size([16, 16])
True
torch.Size([16])
True
torch.Size([2, 16])
True
torch.Size([2])
True
